In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, Input

In [2]:
data= pd.read_csv(r"C:\Users\hsynt\Downloads\Selcuk Abi\Selcuk Abi\VeriSeti2.csv")

In [3]:
# Özellik ve Hedef Ayrımı
X = data.drop(columns=['Target_10', 'adjclose'])
y = data['Target_10']

# Zaman Serisi Formatına Getirme
def create_sequences(data, target, seq_length):
    sequences, targets = [], []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i + seq_length])
        targets.append(target[i + seq_length])
    return np.array(sequences), np.array(targets)

SEQ_LENGTH = 10
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_seq, y_seq = create_sequences(X_scaled, y.values, SEQ_LENGTH)

# PCA ve LDA Veri Hazırlığı
X_flat = X_seq.reshape(X_seq.shape[0], -1)  # Düzleştirilmiş veri

pca = PCA(n_components=10)
X_pca = pca.fit_transform(X_flat)

lda = LDA(n_components=1)
X_lda = lda.fit_transform(X_flat, y_seq)

# GRU Modeli Tanımlama
def build_gru_model(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        GRU(64, activation='relu', return_sequences=False),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# K-Fold Cross Validation
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Eğitim ve Test Sürelerini Hesaplama
results = []

for reduction, X_data in zip(
    ['No PCA/LDA', 'PCA', 'LDA'],
    [X_seq, X_pca, X_lda]
):
    fold_num = 1
    for train_idx, test_idx in kfold.split(X_data, y_seq):
        # Eğitim ve Test Bölünmesi
        X_train, X_test = X_data[train_idx], X_data[test_idx]
        y_train, y_test = y_seq[train_idx], y_seq[test_idx]

        if reduction == 'No PCA/LDA':
            X_train = X_train.reshape(X_train.shape[0], SEQ_LENGTH, -1)
            X_test = X_test.reshape(X_test.shape[0], SEQ_LENGTH, -1)
        else:
            X_train = np.tile(X_train, (1, SEQ_LENGTH)).reshape(X_train.shape[0], SEQ_LENGTH, -1)
            X_test = np.tile(X_test, (1, SEQ_LENGTH)).reshape(X_test.shape[0], SEQ_LENGTH, -1)

        # Modeli Oluştur ve Eğit
        gru_model = build_gru_model(X_train.shape[1:])

        # Eğitim Süresi Ölçümü
        start_time = time.time()
        gru_model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)
        train_time = time.time() - start_time

        # Test Süresi Ölçümü
        start_time = time.time()
        gru_model.predict(X_test)
        test_time = time.time() - start_time

        results.append({
            'Reduction': f"{reduction} (Fold {fold_num})",
            'Model': 'GRU',
            'Train Time (s)': train_time,
            'Test Time (s)': test_time
        })

        fold_num += 1

# Performans Sonuçlarını Görselleştirme
results_df = pd.DataFrame(results)

# Eğitim ve Test Süreleri Tablosu
print("Eğitim ve Test Süreleri Tablosu:")
print(results_df)

# Sonuçları Kaydetme
results_df.to_csv("gru_kfold_train_test_times.csv", index=False)
print("\nSonuçlar 'gru_kfold_train_test_times.csv' dosyasına kaydedildi.")

Epoch 1/10
1041/1041 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8119 - loss: 0.4215
Epoch 2/10
1041/1041 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8686 - loss: 0.3184
Epoch 3/10
1041/1041 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8738 - loss: 0.3011
Epoch 4/10
1041/1041 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8809 - loss: 0.2884
Epoch 5/10
1041/1041 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8794 - loss: 0.2918
Epoch 6/10
1041/1041 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8813 - loss: 0.2872
Epoch 7/10
1041/1041 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8856 - loss: 0.2768
Epoch 8/10
1041/1041 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8886 - loss: 0.2746
Epoch 9/10
1041/1041 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8906 - loss: 0.2693
Epoch 10/10
1041/1041 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8908 - loss: 0.2711
521/521 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Epoch 1/10
1041/1041 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8177 - los